In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

In [2]:
PM10 = pd.read_csv('./data/PM10-2023.csv', sep=';')
NO2 = pd.read_csv('./data/NO2-2023.csv', sep=';')
traffic = pd.DataFrame()
for file in os.listdir('./data/natezenie_ruchu_2023'):
    data = pd.read_csv('./data/natezenie_ruchu_2023/' + file, sep=';')
    traffic = pd.concat([traffic, data])

In [3]:
PM10 = PM10.drop_duplicates()
traffic = traffic.drop_duplicates()
NO2 = NO2.drop_duplicates()

In [4]:
traffic.columns = ['Traffic_station_id', 'date', 'hour', 'quality', 'num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed', 'num_trucks', 'avg_truck_speed']
df = pd.read_excel('./data/Stammdaten_Verkehrsdetektion_2022_07_20.xlsx')
# a = []
# for i, item in enumerate(df['DET_ID15']):
#     if item in traffic['station_id'].unique():
#         a.append([df['x'][i], df['y'][i]])

In [5]:
traffic_id = [100101010031590, 100101010031287]
traffic['date'] = pd.to_datetime(traffic['date'], format="%d.%m.%Y")
traffic['hour'] = pd.to_timedelta(traffic['hour'], unit='h')
traffic['datetime'] = traffic['date'] + traffic['hour']
traffic = traffic.drop(columns=['date', 'hour'])
traffic = traffic[traffic['Traffic_station_id'].isin(traffic_id)]

In [6]:
PM10['Date'] = pd.to_datetime(PM10['Date'], format="%Y-%m-%d")
PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
PM10['Time'][PM10['Time'] == '24:00'] = '00:00'
PM10['Time'] = pd.to_timedelta(PM10['Time'].str.strip("'") + ':00')
PM10['datetime'] = PM10['Date'] + PM10['Time']
PM10 = PM10.drop(columns=['Date', 'Time', "State / Measuring network", "Station setting","Station type","Pollutant","Time scope", "Unit", "Station name"])

C:\Users\wpietron\AppData\Local\Temp\ipykernel_16148\72315281.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Date'][PM10['Time'] == '24:00'] = PM10['Date'] + pd.Timedelta(days=1)
C:\Users\wpietron\AppData\Local\Temp\ipykernel_16148\72315281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PM10['Time'][PM10['Time'] == '24:00'] = '00:00'


In [7]:
pivot_PM10 = PM10.pivot_table(index=['datetime'],
                          columns=['Station code'],
                          values=['Measure value'],
                          aggfunc='first')

In [8]:
pivot_PM10.columns = ['{}_{}'.format(col[1], col[0]) for i, col in enumerate(pivot_PM10.columns.values)]
pivot_PM10.reset_index(inplace=True)
# pivot_PM10.to_csv("pivot_PM.csv", index=False)

In [9]:
pivot_traffic = traffic.pivot_table(index=['datetime'],
                          columns=['Traffic_station_id'],
                          values=['num_vehicles', 'avg_speed', 'num_cars', 'avg_car_speed',
                                  'num_trucks', 'avg_truck_speed','quality'],
                          aggfunc='first')

In [10]:
pivot_traffic.columns = ['{}_{}'.format(col[1], col[0]) for i, col in enumerate(pivot_traffic.columns.values)]
pivot_traffic.reset_index(inplace=True)
# pivot_traffic.to_csv("pivot_traffic.csv", index=False)

In [11]:
pivot_traffic = pivot_traffic[pivot_traffic['datetime'].isin(pivot_PM10['datetime'])]
pivot_PM10 = pivot_PM10[pivot_PM10['datetime'].isin(pivot_traffic['datetime'])]

pivot_traffic = pivot_traffic.sort_values(by='datetime')
pivot_PM10 = pivot_PM10.sort_values(by='datetime')

In [12]:
#save to csv
pivot_traffic.to_csv("pivot_traffic.csv", index=False)
pivot_PM10.to_csv("pivot_PM10.csv", index=False)